In [ ]:

import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances, merge_chunks_unite_instances2, remove_semantics, get_merge_pcds, merge_unite_gt
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels,generate_random_colors_map
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics.metrics_class import Metrics
import matplotlib.pyplot as plt 
import shutil
from tqdm import tqdm 

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/media/cedric/Datasets2/semantic_kitti/')
end_inds = {0:4541,1:1100,2:4661,3:800,4:271,5:2761,6:1101,7:1100,8:4071,9:1591,10:1201}
SEQUENCE_NUM = 7
old = False 

ind_start = 0
ind_end = end_inds[SEQUENCE_NUM]
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False

out_chunks = '../../pcd_preprocessed/output_chunks/'

out_folder_ncuts = out_chunks + 'test_data' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_folder_ncuts) == False:
        os.makedirs(out_folder_ncuts)

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = '../../pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)
## This cell can be ignored after first run as outputs are stored 

In [ ]:
if os.path.exists(out_folder + 'all_poses_' + str(SEQUENCE_NUM) + '_' + str(0) + '.npz') == False:
        process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                                major_voxel_size=major_voxel_size,icp=False,
                                out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                                ground_segmentation_method=ground_segmentation_method)

This cell can be ignored after first run as outputs are stored 

In [ ]:
##load data if already stored 

if os.path.exists(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd') == False:
        pcd_ground_minor, pcd_nonground_minor,\
                all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                        ground_mode=ground_segmentation_method)
        #o3d.visualization.draw_geometries([color_pcd_by_labels(pcd_nonground_minor,kitti_labels['seg_nonground'])])
        o3d.io.write_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd', pcd_ground_minor, write_ascii=False, compressed=False, print_progress=False)
        o3d.io.write_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd', pcd_nonground_minor, write_ascii=False, compressed=False, print_progress=False)
        np.savez(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz',
                                                instance_nonground=kitti_labels['instance_nonground'],
                                                instance_ground=kitti_labels['instance_ground'],
                                                seg_ground = kitti_labels['seg_ground'],
                                                seg_nonground=kitti_labels['seg_nonground']
                                                )
        #o3d.visualization.draw_geometries([pcd_nonground_minor])


In [ ]:
pcd_ground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_ground_minor{SEQUENCE_NUM}_0.pcd')
pcd_nonground_minor = o3d.io.read_point_cloud(f'{out_folder}pcd_nonground_minor{SEQUENCE_NUM}_0.pcd')
print(pcd_ground_minor)
kitti_labels_orig = {}
with np.load(f'{out_folder}kitti_labels_preprocessed{SEQUENCE_NUM}_0.npz') as data :
        kitti_labels_orig['instance_ground'] = data['instance_ground']
        kitti_labels_orig['instance_nonground'] = data['instance_nonground']
        kitti_labels_orig['seg_nonground'] = data['seg_nonground']
        kitti_labels_orig['seg_ground'] = data['seg_ground']

        

with np.load(f'{out_folder}all_poses_{SEQUENCE_NUM}_0.npz') as data:
        all_poses = data['all_poses']
        T_pcd = data['T_pcd']
        first_position = T_pcd[:3, 3]

In [ ]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

Now we subsample the poses based on a voxel_size

In [ ]:
if os.path.exists(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz') == False : 
	print(f'{out_folder}subsampled_data{str(SEQUENCE_NUM)}_0.npz')
	poses, positions, \
	sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start,sequence_num=SEQUENCE_NUM,out_folder=out_folder)



In [ ]:
with np.load(f'{out_folder}subsampled_data{SEQUENCE_NUM}_0.npz') as data:
	poses=data['poses']
	positions=data['positions']
	sampled_indices_local = data['sampled_indices_local']
	sampled_indices_global=data['sampled_indices_global']

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels, obbs = chunk_and_downsample_point_clouds(dataset,pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels_orig)

In [ ]:
def color_pcd_by_labels(pcd, labels,colors=None,gt_labels=None,semantics=False):
    
    if colors == None : 
        colors = generate_random_colors(2000)
    pcd_colored = copy.deepcopy(pcd)
    pcd_colors = np.zeros(np.asarray(pcd.points).shape)
    if gt_labels is None :
    	unique_labels = list(np.unique(labels)) 
    else: 
        unique_labels = list(np.unique(gt_labels))
    
    background_color = np.array([0,0,0])
    #for i in range(len(pcd_colored.points)):
    for i in unique_labels:
        if i == -1 : 
            continue
        idcs = np.where(labels == i)
        idcs = idcs[0]
        if i == 0 and semantics == False : 
            pcd_colors[idcs] = background_color
        else : 
            pcd_colors[idcs] = np.array(colors[unique_labels.index(i)])
        
    if semantics : 
        pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors)
    else : 
        pcd_colored.colors = o3d.utility.Vector3dVector(pcd_colors/255)
    return pcd_colored

In [ ]:
color_dict_normalized = {
    0: [0.0, 0.0, 0.0],
    1: [0.0, 0.0, 1.0],
    10: [0.9607843137254902, 0.5882352941176471, 0.39215686274509803],
    11: [0.9607843137254902, 0.9019607843137255, 0.39215686274509803],
    13: [0.9803921568627451, 0.3137254901960784, 0.39215686274509803],
    15: [0.5882352941176471, 0.23529411764705882, 0.11764705882352941],
    16: [1.0, 0.0, 0.0],
    18: [0.7058823529411765, 0.11764705882352941, 0.3137254901960784],
    20: [1.0, 0.0, 0.0],
    30: [0.11764705882352941, 0.11764705882352941, 1.0],
    31: [0.7843137254901961, 0.1568627450980392, 1.0],
    32: [0.35294117647058826, 0.11764705882352941, 0.5882352941176471],
    40: [1.0, 0.0, 1.0],
    44: [1.0, 0.5882352941176471, 1.0],
    48: [0.29411764705882354, 0.0, 0.29411764705882354],
    49: [0.29411764705882354, 0.0, 0.6862745098039216],
    50: [0.0, 0.7843137254901961, 1.0],
    51: [0.19607843137254902, 0.47058823529411764, 1.0],
    52: [0.0, 0.5882352941176471, 1.0],
    60: [0.6666666666666666, 1.0, 0.5882352941176471],
    70: [0.0, 0.6862745098039216, 0.0],
    71: [0.0, 0.23529411764705882, 0.5294117647058824],
    72: [0.3137254901960784, 0.9411764705882353, 0.5882352941176471],
    80: [0.5882352941176471, 0.9411764705882353, 1.0],
    81: [0.0, 0.0, 1.0],
    99: [1.0, 1.0, 0.19607843137254902],
    252: [0.9607843137254902, 0.5882352941176471, 0.39215686274509803],
    256: [1.0, 0.0, 0.0],
    253: [0.7843137254901961, 0.1568627450980392, 1.0],
    254: [0.11764705882352941, 0.11764705882352941, 1.0],
    255: [0.35294117647058826, 0.11764705882352941, 0.5882352941176471],
    257: [0.9803921568627451, 0.3137254901960784, 0.39215686274509803],
    258: [0.7058823529411765, 0.11764705882352941, 0.3137254901960784],
    259: [1.0, 0.0, 0.0]
}

reverse_color_dict = {tuple(v): k for k, v in color_dict_normalized.items()}


def color_pcd_kitti(pcd,labels): 
        unique_labels = np.unique(labels)
        pcd_colors = np.zeros_like(np.asarray(pcd.points))
        for i in unique_labels:
                idcs = np.where(labels == i)
                idcs = idcs[0]
                pcd_colors[idcs] = np.array(color_dict_normalized[int(i)])
        
        pcd.colors = o3d.utility.Vector3dVector(pcd_colors)
        return pcd 

In [ ]:
def calc_num_instance(sem_classes,labels_instances,labels_sem):
    num_inst = len(np.unique(labels_instances)) - 1

    num_sem_inst = len(np.unique(labels_sem[np.isin(labels_sem, sem_classes)]))

    num_whole_inst = num_inst + num_sem_inst
    return num_whole_inst, labels_sem
    
def get_semantic_map(inst_labels, sem_labels, sem_classes=[49, 50, 51, 70, 72]):

    semantic_map = {}
    total_merges = 0

    inst_labels = inst_labels[np.isin(sem_labels, sem_classes)]
    sem_labels = sem_labels[np.isin(sem_labels, sem_classes)]
    for inst in np.unique(inst_labels)[1:]:
        intersect, area = np.unique(sem_labels[inst_labels == inst], return_counts=True)
        main_sem_label = intersect[np.argmax(area)]
        semantic_map[inst] = main_sem_label
        to_merge_sem_labels = intersect[intersect != main_sem_label]
        num_merges = len(to_merge_sem_labels)
        total_merges += num_merges
    return semantic_map
    
def get_num_splits(intersection, sem_classes, t_area, t_score, t_ratio):
    result = {
        "inst": [],
        "area": [],
        "score": [],
        "sem_labels": [],
        "num_splits": [],
        "sem_splits": [],
        "main_sem_label": [],
        "other_splits": [],
        "valid_sem_label": [],
        "sem-sem_splits": [],
    }
    for idx, inst in enumerate(intersection["inst"]):
        # print(f"inst: {inst}, area: {intersection['area'][inst]}, score: {intersection['score'][inst]}, sem_labels: {intersection['sem_labels'][inst]}, num_intersection: {intersection['num_intersection'][inst]}")
        area = np.sum(intersection["area"][idx])
        score = intersection["score"][idx]
        sem_label = intersection["sem_labels"][idx]
        if area > t_area and score < t_score and inst != 0:
            intersect_area = intersection["area"][idx]
            num_splits = np.count_nonzero(intersect_area / area > t_ratio) - 1
            valid_sem_label = sem_label[intersect_area / area > t_ratio]
            sem_sem_splits = np.count_nonzero(np.isin(valid_sem_label, sem_classes)) - 1
            other_splits = num_splits - sem_sem_splits
            result["inst"].append(inst)
            result["area"].append(area)
            result["score"].append(score)
            result["sem_labels"].append(sem_label)
            result["num_splits"].append(num_splits)
            result["sem_splits"].append(sem_label[intersect_area / area > t_ratio])
            result["main_sem_label"].append(sem_label[np.argmax(intersect_area)])
            result["other_splits"].append(other_splits)
            result["valid_sem_label"].append(valid_sem_label)
            result["sem-sem_splits"].append(sem_sem_splits)
        num_splits = np.sum(result["num_splits"])
        sem2sem_splits = np.sum(result["sem-sem_splits"])
        # print(f"inst: {inst}, area: {area}, score: {score}, sem_labels: {intersection['sem_labels'][idx]}, num_intersection: {intersection['num_intersection'][idx]}, num_splits: {num_splits}")
    return result, (num_splits, sem2sem_splits)
    

def create_sub_instances(inst_labels, sem_labels, result):
    new_inst_labels = inst_labels.copy()
    last_inst_id = np.max(inst_labels)
    for inst in np.unique(inst_labels):

        if inst in result["inst"]:
            idx = np.where(result["inst"] == inst)[0][0]
            sem_splits = result["sem_splits"][idx]
            main_sem_label = result["main_sem_label"][idx]
            to_split = sem_splits[sem_splits != main_sem_label]
            for split in to_split:
                last_inst_id += 1
                # print(f"Splitting instance {inst} with sem label {split} into {last_inst_id}")
                new_inst_labels[
                    np.logical_and(inst_labels == inst, sem_labels == split)
                ] = last_inst_id
    return new_inst_labels
    
def get_intersection(labels_sem, pred_labels, ignore_labels=[0, 1, 52, 99, 40, 48, 49]):
    intersection = {
        "inst": [],
        "sem_labels": [],
        "area": [],
        "num_intersection": [],
        "score": [],
    }
    for i in ignore_labels:
        labels_sem[labels_sem == i] = 0
    pred_labels = pred_labels[labels_sem != 0]
    labels_sem = labels_sem[labels_sem != 0]
    for label in np.unique(pred_labels):
        intersect, area = np.unique(
            labels_sem[pred_labels == label], return_counts=True
        )
        intersection["inst"].append(label)
        intersection["sem_labels"].append(intersect)
        intersection["area"].append(area)
        intersection["num_intersection"].append(len(intersect))
        max_area = np.max(area)
        # error_area = np.sum(area[area != max_area])
        score = max_area / np.sum(area)
        intersection["score"].append(score)
    return intersection

In [ ]:
import sklearn
from sklearn.cluster import DBSCAN, HDBSCAN
import hdbscan

#cvc clustering setup 
#params = [2,0.4,1.5]


def uniform_down_sample_with_indices(points, every_k_points):
        # Create a new point cloud for the downsampled output

        # List to hold the indices of the points that are kept
        indices = []

        # Iterate over the points and keep every k-th point
        for i in range(0, points.shape[0], every_k_points):
            indices.append(i)

        return indices

def downsample_chunk(points):
        num_points_to_sample = 30000
        every_k_points = int(
            points.shape[0] /
            num_points_to_sample)
        indeces = uniform_down_sample_with_indices(
            points, every_k_points)


        return points[indeces]

def clustering_logic(pcd_nonground_chunk, pcd_ground_chunk,
                        eps=0.3, min_samples=10,method='hdbscan'):
    """
    Perform DBSCAN clustering on the point cloud data.

    :param cur_pcd: Current point cloud for clustering.
    :param pcd_all: All point cloud data.
    :param eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
    :param min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
    :return: Cluster labels for each point in the point cloud.
    """
    
    inliers = get_statistical_inlier_indices(pcd_ground_chunk)
    ground_inliers = get_subpcd(pcd_ground_chunk, inliers)
    mean_hight = np.mean(np.asarray(ground_inliers.points)[:,2])
    in_idcs = np.where(np.asarray(ground_inliers.points)[:,2] < (mean_hight + 0.2))[0]
    cut_hight = get_subpcd(ground_inliers, in_idcs)
    cut_hight.paint_uniform_color([0, 0, 0])
    
    #in_idcs = np.where(np.asarray(pcd_nonground_chunk.points)[:,2] > (mean_hight + 0.05))[0]
    #pcd_nonground_corrected = get_subpcd(pcd_nonground_chunk, in_idcs)
    pcd_nonground_corrected = pcd_nonground_chunk
    
    pcd_nonground_downsampled = o3d.geometry.PointCloud()
    pts_downsampled = downsample_chunk(np.asarray(pcd_nonground_corrected.points))
    pcd_nonground_downsampled.points = o3d.utility.Vector3dVector(pts_downsampled)
    
    #clustering = DBSCAN(eps=eps, min_samples=min_samples)
    #clustering = HDBSCAN(min_cluster_size=10).fit(pts_downsampled)
    if method == 'hdbscan': 
        clustering = hdbscan.HDBSCAN(algorithm='best', alpha=1., approx_min_span_tree=True,
                                    gen_min_span_tree=True, leaf_size=100,
                                    metric='euclidean', min_cluster_size=10, min_samples=None
                                )
        clustering.fit(pts_downsampled)
        
        labels_not_road = clustering.labels_

        #labels_not_road = np.asarray(cluster_indices) 
        
    colors_gen = generate_random_colors(5000)
    
    labels_not_road = labels_not_road + 1
    # Reproject cluster labels to the original point cloud size
    cluster_labels = np.ones((len(pcd_nonground_corrected.points), 1)) * -1
    labels_non_ground = kDTree_1NN_feature_reprojection(cluster_labels, pcd_nonground_corrected, labels_not_road.reshape(-1,1), pcd_nonground_downsampled )
    colors = np.zeros((labels_non_ground.shape[0],3))
    unique_labels = list(np.unique(labels_non_ground))
    
    for j in unique_labels:
            cur_idcs = np.where(labels_non_ground == j)[0]
            if j == 0 : 
                colors[cur_idcs] = np.array([0,0,0])
            else : 
                colors[cur_idcs] = np.array(colors_gen[unique_labels.index(j)])
        
    pcd_nonground_corrected.colors = o3d.utility.Vector3dVector(colors / 255.)
    
    #o3d.visualization.draw_geometries([pcd_nonground_corrected])
    
    
    
    return pcd_nonground_corrected, cut_hight, inliers, in_idcs

In [ ]:
from collections import Counter

alpha = 1.0
theta = 0.5
colors = generate_random_colors_map(600)
beta = 0.0
gamma = 0.1
proximity_threshold = 1.0
tarl_norm = False
ncuts_thresholds = [0.0025,0.005,0.01,0.02]
        
out_name = 'tarl_dino_spatial' + str(SEQUENCE_NUM) 
# Generate 30 different colors
 

COLORS = generate_random_colors(400)
COLORS = [(col[0]/255.,col[1]/255.,col[2]/255.) for col in COLORS]


out_kitti = out_chunks + 'out_kitti' + str(SEQUENCE_NUM) + '/'
if os.path.exists(out_kitti) == True : 
        shutil.rmtree(out_kitti)

os.makedirs(out_kitti)
        
out_kitti_instance = out_chunks + 'out_kitti_instance' + str(SEQUENCE_NUM) + '/'


if os.path.exists(out_kitti_instance) == False : 
        os.makedirs(out_kitti_instance)


out_kitti_semantic = out_chunks + 'out_kitti_semantic' + str(SEQUENCE_NUM) + '/'

if os.path.exists(out_kitti_semantic) == False : 
        os.makedirs(out_kitti_semantic)


limit = -1 ##use this for experiments to run limit chunks numberss

sem_classes = [49, 50, 51, 70, 72]
ignore_labels = [0, 1, 52, 99, 40, 48, 49]


patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
semantics = np.hstack((kitti_labels_orig['seg_nonground'].reshape(-1,),kitti_labels_orig['seg_ground'].reshape(-1,)))

instances = np.hstack((kitti_labels_orig['instance_nonground'].reshape(-1,),kitti_labels_orig['instance_ground'].reshape(-1,)))
                
merge_pcd = o3d.geometry.PointCloud()    

print("total sequence number",len(center_ids))
for ncuts_threshold in ncuts_thresholds: 
        print("using threshold",ncuts_threshold)
        for sequence in tqdm(range(1,2)):
                        '''
                        merged_chunk,file_name, pcd_chunk, pcd_chunk_ground,inliers, inliers_ground = ncuts_chunk(dataset,list(indices),pcd_nonground_chunks,pcd_ground_chunks,
                                pcd_nonground_chunks_major_downsampling,
                                pcd_nonground_minor,T_pcd,center_positions,center_ids,
                                positions,first_position,list(sampled_indices_global),
                                chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                                alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                                proximity_threshold=proximity_threshold,
                                out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                                patchwise_indices=patchwise_indices,ncuts_threshold=ncuts_threshold,obb=obbs[sequence])
                        '''
                        pcd_chunk, pcd_chunk_ground, inliers, inliers_ground = clustering_logic(pcd_nonground_chunks[sequence],pcd_ground_chunks[sequence],
                        eps=0.4, min_samples=10,method='hdbscan')
        
                        
                        #o3d.visualization.draw_geometries([merged_chunk])
                        
                        seg_ground = kitti_labels['ground']['semantic'][sequence][inliers][inliers_ground]
                        inst_ground = kitti_labels['ground']['instance'][sequence][inliers][inliers_ground]
                        
                        file_name = str(center_ids[sequence]).zfill(6) + '.pcd'
        
                        kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['semantic'][sequence].reshape(-1,),
                                                colors=COLORS,gt_labels=semantics,semantics=True
                                                )
                        
                        kitti_chunk_instance = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['instance'][sequence].reshape(-1,),
                                                colors=colors,gt_labels=instances)
                                                
                        kitti_chunk_instance_ground = color_pcd_by_labels(pcd_chunk_ground,inst_ground.reshape(-1,),
                                                colors=colors,gt_labels=instances)
                                                
                        kitti_chunk_semantic_ground = color_pcd_by_labels(pcd_chunk_ground,seg_ground.reshape(-1,),
                                                colors=COLORS,gt_labels=semantics,semantics=True)
                        
                        #o3d.visualization.draw_geometries([kitti_chunk_instance + kitti_chunk_instance_ground])
                        #o3d.visualization.draw_geometries([pcd_chunk + pcd_chunk_ground])
                        #semantic_labels = np.hstack(( kitti_labels['nonground']['semantic'][sequence].reshape(-1,),seg_ground.reshape(-1,)))
                        #np.savez(out_kitti_semantic + file_name.split('.')[0] + '.npz',labels=semantic_labels)
                        semantics_non_ground = color_pcd_kitti(copy.deepcopy(pcd_chunk),kitti_labels['nonground']['semantic'][sequence].reshape(-1,))
                        semantics_ground = color_pcd_kitti(copy.deepcopy(pcd_chunk_ground),seg_ground.reshape(-1,))
                        kitti_semantics = np.hstack((kitti_labels['nonground']['semantic'][sequence].reshape(-1,),seg_ground.reshape(-1,)))
                        
                        kitti_instance = kitti_chunk_instance + kitti_chunk_instance_ground
                        pcd_merged = pcd_chunk + pcd_chunk_ground
                        
                        unique_colors, labels_kitti = np.unique(np.asarray(kitti_instance.colors),axis=0, return_inverse=True)
                        unique_colors, labels_ncuts = np.unique(np.asarray(pcd_merged.colors),axis=0, return_inverse=True)
                        
                        new_ncuts_labels = remove_semantics(labels_kitti,copy.deepcopy(labels_ncuts))
                        metrics_ncuts = Metrics(name='ncuts with threshold ' + str(ncuts_threshold) )
                        metrics_ncuts.update_stats(labels_ncuts,new_ncuts_labels,labels_kitti)
                        
                        
                        ##semantic stuff 
                        num_instance, sem_labels = calc_num_instance(sem_classes,labels_kitti,kitti_semantics)
                        
                        intersection = get_intersection(sem_labels, labels_ncuts)
                        print('intersection')
                        print(intersection)
                        result, (num_splits, sem2sem_splits) = get_num_splits(intersection, sem_classes, 200, 0.9, 0.02)
                        new_labels = create_sub_instances(labels_ncuts, sem_labels, result)
                        semantic_map = get_semantic_map(new_labels, sem_labels)
                        value_counts = Counter(semantic_map.values())
                        num_merges = np.sum(list(value_counts.values())) - len(value_counts)
                        
                        print('number of splits',num_splits)
                        print('sem sem splits', sem2sem_splits)
                        print('num merges', num_merges)
                        print('---------------------')
                        
                        #o3d.visualization.draw_geometries([kitti_instance])
                        #o3d.visualization.draw_geometries([semantics_ground + semantics_non_ground])
                        o3d.visualization.draw_geometries([pcd_merged])